# An Analysis of Air Quality in India

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from scipy import stats
import scipy

In [ ]:
aqi = pd.read_csv("city_day.csv")
aqi

In [ ]:
aqi.drop(columns = ["City", "Date", "AQI_Bucket"], inplace = True)
aqi = aqi.dropna().reset_index(drop = True)
aqi

In [ ]:
pd.plotting.scatter_matrix(aqi, diagonal = "None", alpha = 1,figsize = (15,15))
plt.savefig("scatter_matrix.png", bbox_inches = "tight")
plt.show()

Check Multicollinearity

In [ ]:
#drop outliers
aqi = aqi[aqi["AQI"] <= 400]
aqi = aqi.reset_index(drop = True)

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
#Get ANOVA table
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 12
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

In [ ]:
aqi.corr()

Drop PM2.5 and NOx for multicolinearity

In [ ]:
aqi.drop(columns = ["PM2.5", "NOx"], inplace = True)
aqi

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
vif_df = pd.DataFrame()
vif_df["variables"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values,i) for i in range(len(X.columns))]
vif_df

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
#Get ANOVA table
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 10
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

Now we will do variable selection, This was done in R. 

In [ ]:
aqi.drop(columns = ["NO2","Benzene"], inplace = True)

In [ ]:
X = aqi.iloc[:, 0:-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 8
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

In [ ]:
wil = model.params.to_list()

Now look at the residuals of the model. Get Studentized residuals

In [ ]:
influence = model.get_influence()

In [ ]:
#residual plot, to check GMA
plt.scatter(model.fittedvalues, influence.resid_studentized_external)
plt.xlabel("y_hat")
plt.ylabel("studentized residuals")
plt.title("studentzized residuals vs. y_hat")
plt.savefig("residual_plot1.png", bbox_inches = "tight")
plt.show()

In [ ]:
#Get leverage and influential points
leverage = influence.hat_matrix_diag
dffits = influence.dffits
lev_tol = 2*(11/len(aqi))
dffits_tol = 2*np.sqrt(11/(len(aqi)))
lev_index = []
for i in range(0,len(aqi)):
    if leverage[i] > lev_tol:
        lev_index.append(i)
dffits_index = []
for i in range(0,len(aqi)):
    if np.abs(dffits[0][i]) > dffits_tol:
        dffits_index.append(i)
        
indeces = lev_index + dffits_index
len(indeces)

aqi.drop(indeces, axis = 0, inplace = True)
aqi = aqi.reset_index(drop = True)

X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 8
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

In [ ]:
influence = model.get_influence()
plt.scatter(model.fittedvalues, influence.resid_studentized_external)
plt.xlabel("y_hat")
plt.ylabel("studentized residuals")
plt.title("studentzized residuals vs. y_hat")
plt.savefig("residual_plot2.png", bbox_inches = "tight")
plt.show()

In [ ]:
mil = model.params.to_list()

Check change in coefficients after removal of leverage and influential points.

In [ ]:
diffil = np.array(wil) - np.array(mil)
pc = np.abs(diffil/wil)

In [ ]:
diffil

In [ ]:
pc*100

Now we will put the PM2.5 and NOx back in and drop the PM10 and NO

In [ ]:
aqi = pd.read_csv("city_day.csv")
aqi.drop(columns = ["City", "Date", "AQI_Bucket"], inplace = True)
aqi = aqi[aqi["AQI"] <= 400]
aqi = aqi.dropna().reset_index(drop = True)
aqi.drop(columns = ["PM10", "NO"], inplace = True)
aqi

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
vif_df = pd.DataFrame()
vif_df["variables"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values,i) for i in range(len(X.columns))]
vif_df

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 10
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

Variable selection done in R

In [ ]:
aqi.drop(columns = ["SO2", "Toluene"], inplace = True)
aqi

In [ ]:
X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 8
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

In [ ]:
wil = model.params.to_list()

In [ ]:
influence = model.get_influence()

In [ ]:
plt.scatter(model.fittedvalues, influence.resid_studentized_external)
plt.xlabel("y_hat")
plt.ylabel("studentized residuals")
plt.title("studentzized residuals vs. y_hat")
plt.savefig("residual_plot3.png", bbox_inches = "tight")
plt.show()

In [ ]:
leverage = influence.hat_matrix_diag
dffits = influence.dffits
lev_tol = 2*(11/len(aqi))
dffits_tol = 2*np.sqrt(11/(len(aqi)))
lev_index = []
for i in range(0,len(aqi)):
    if leverage[i] > lev_tol:
        lev_index.append(i)
dffits_index = []
for i in range(0,len(aqi)):
    if np.abs(dffits[0][i]) > dffits_tol:
        dffits_index.append(i)
        
indeces = lev_index + dffits_index
len(indeces)

aqi.drop(indeces, axis = 0, inplace = True)
aqi = aqi.reset_index(drop = True)

X = aqi.iloc[:, :-1]
y = aqi.iloc[:,-1]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

In [ ]:
ybar = np.mean(y)
SST = np.sum((y - ybar)**2)
SSR = np.sum((model.fittedvalues - ybar)**2)
SSE = np.sum(model.resid**2)
k = 8
n = len(aqi)
dfR = k
dfE = n - k - 1
dfT = n - 1
MSE = SSE/dfE
MSR = SSR/dfR
F_0  = MSR/MSE
p_value = 1 - scipy.stats.f.cdf(F_0, dfR, dfE)
columns = ["Source", "df", "MS", "F_0", "pvalue"]
index = ["Regression", "Error", "Total"]
data = [[SSR, dfR, MSR, F_0, float(p_value)],[SSE, dfE, MSE,  ],[SST, dfT, ]]
anova = pd.DataFrame(data, index, columns)
anova

In [ ]:
influence = model.get_influence()
plt.scatter(model.fittedvalues, influence.resid_studentized_external)
plt.xlabel("y_hat")
plt.ylabel("studentized residuals")
plt.title("studentzized residuals vs. y_hat")
plt.savefig("residual_plot4.png", bbox_inches = "tight")
plt.show()

In [ ]:
mil = model.params.to_list()

In [ ]:
diffil = np.array(wil) - np.array(mil)
pc = np.abs(diffil/wil)

In [ ]:
diffil

In [ ]:
pc*100